In [ ]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-2.1.0+cu118.html
!pip install torch-geometric
!pip install ogb

In [2]:
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset
import torch
dataset_name = 'ogbg-molhiv'
# Load the dataset and transform it to sparse tensor
dataset = PygGraphPropPredDataset(name=dataset_name,
                                transform=T.ToSparseTensor())
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

# Extract the graph
print(dataset[0])
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# If you use GPU, the device should be cuda
print('Device: {}'.format(device))

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:02<00:00,  1.05it/s]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:02<00:00, 20005.38it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:04<00:00, 9247.06it/s]


Saving...
The ogbg-molhiv dataset has 41127 graph
Data(edge_attr=[40, 3], x=[19, 9], y=[1, 1], num_nodes=19, adj_t=[19, 19, nnz=40])
Device: cuda


Done!


In [33]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout):
        super(GCN, self).__init__()

        # A list of GCNConv layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GCNConv(input_dim, hidden_dim))
        for _ in range(num_layers -2):
          self.convs.append(GCNConv(hidden_dim, hidden_dim))
        self.convs.append(GCNConv(hidden_dim, output_dim))

        # A list of 1D batch normalization layers
        self.bns = torch.nn.ModuleList()
        for _ in range(num_layers - 1):
          self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
        self.dropout = dropout

        self.linear = torch.nn.Linear(output_dim, 1)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t, batch):

        out = None

        for i in range(len(self.convs) - 1):
          x = self.convs[i](x, adj_t)
          x = self.bns[i](x)
          x = F.relu(x)
          x = F.dropout(x, self.dropout, training=self.training)
        x = self.convs[len(self.convs) - 1](x, adj_t)
        x = global_mean_pool(x, batch)

        x = self.linear(x)

        out = x

        return out

In [34]:
import torch
from torch_geometric.data import DataLoader
from torch_geometric.nn import GCNConv
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.datasets import OGB_MAG
import numpy as np
from torch_geometric.nn.pool import global_mean_pool
from tqdm import tqdm

split_idx = dataset.get_idx_split()
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=64, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=64, shuffle=False)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=64, shuffle=False)

# Define the GCN model
# class GCNModel(torch.nn.Module):
#     def __init__(self, in_channels, hidden_channels, out_channels):
#         super(GCNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, out_channels)
#         self.linear = Linear(out_channels, 1)
#         self.sigmoid = torch.nn.Sigmoid()

#     def forward(self, x, edge_index, batch):
#         x = self.conv1(x, edge_index)
#         x = F.relu(x)
#         x = self.conv2(x, edge_index)
#         x = F.relu(x)
#         x = global_add_pool(x, batch)  # Global pooling operation (sum in this case)
#         x = self.linear(x)
#         x = self.sigmoid(x)
#         return x

# model = GCNModel(in_channels=dataset.num_node_features,
#                  hidden_channels=64,
#                  out_channels=64).to(device)
args = {
      'device': device,
      'num_layers': 5,
      'hidden_dim': 256,
      'dropout': 0.5,
      'lr': 0.01,
      'epochs': 50,
      "out_channels": 128,
  }

model = GCN(dataset.num_node_features, args['hidden_dim'],
            args["out_channels"], args['num_layers'],
            args['dropout']).to(device)
# evaluator = Evaluator(name='ogbg-molhiv')
# Loss and optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])

# Training loop
def train():
  model.train()
  for epoch in range(args["epochs"]):
      train_loss = []
      for data in tqdm(train_loader, total=len(train_loader)):
          optimizer.zero_grad()
          data = data.to(device)
          data.adj_t = data.adj_t.to_symmetric()
          data.x = data.x.float()
          out = model(data.x, data.adj_t, data.batch)
          loss = criterion(out, data.y.view(-1, 1).to(torch.float32))
          loss.backward()
          optimizer.step()
          train_loss.append(loss.item())

      print(f"Epoch #{epoch + 1}. Loss:{sum(train_loss)/len(train_loss)}")


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [35]:
train()

100%|██████████| 515/515 [00:20<00:00, 25.32it/s]


Epoch #1. Loss:0.16835968867215573


100%|██████████| 515/515 [00:19<00:00, 26.06it/s]


Epoch #2. Loss:0.1575283563976149


100%|██████████| 515/515 [00:20<00:00, 25.62it/s]


...

Epoch #98. Loss:0.12533599470642584


100%|██████████| 515/515 [00:19<00:00, 26.36it/s]


Epoch #99. Loss:0.12485843604629479


100%|██████████| 515/515 [00:19<00:00, 26.14it/s]

Epoch #100. Loss:0.12656981169417936


In [32]:
# Evaluation
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for data in valid_loader:
        data = data.to(device)
        data.x = data.x.float()
        data.adj_t = data.adj_t.to_symmetric()
        out = model(data.x, data.adj_t, data.batch)
        y_true.append(data.y.view(-1).cpu().numpy())
        y_pred.append((out > 0.5).view(-1).cpu().numpy())

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

# Evaluate using appropriate metrics (e.g., accuracy, F1 score, ROC AUC)
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(f"F1 Score: {f1_score(y_true, y_pred)}")
print(f"ROC AUC Score: {roc_auc_score(y_true, y_pred)}")

Accuracy: 0.9807926088013615
F1 Score: 0.20202020202020202
ROC AUC Score: 0.560736331569665


In [ ]:
print(list(y_true).count(0))
print(list(y_true).count(1))

4032
81


In [25]:
for i in range(len(y_true)):
  if y_pred[i] == True:
    print(y_true[i], y_pred[i])

1 True
0 True
0 True
1 True
1 True
1 True
1 True
1 True
1 True
0 True
1 True
1 True
